In [1]:
#importing required libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
 import re
import string

In [3]:
#giving user access
headers=({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Accept-Language':'en-US , en ; q=0.5'})

In [4]:
df = pd.read_csv('20_pages_products.csv')

In [5]:
df.tail()

,Product Name,Product URL,Product Price(₹),Product Rating,Product Number of Reviews
421,Tabelito® VOLT Casual waterproof Laptop Backpa...,https://www.amazon.in/Tabelito%C2%AE-VOLT-wate...,999,4.8,(82)
422,WENGER CITYUPGRADE 16 Inch Laptop Backpack wit...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"5,985",4.4,(131)
423,Tommy Hilfiger Jaylen Unisex Polyester 15 Inch...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"2,121.",5.0,(1)
424,Lavie Sport Chairman Laptop Bag | Premium Leat...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,899,4.0,(727)
425,DIKTOO 38L Large Laptop Backpack with 2 Compar...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,799,NaN,NaN


In [6]:
urls = df.iloc[:,1]

In [7]:
print(urls[8])

https://www.amazon.in/GEAR-Black-Blue-Backpack-years/dp/B019HA8AYG/ref=sr_1_9?crid=2M096C61O4MLT&keywords=bags&qid=1679481857&sprefix=ba%2Caps%2C283&sr=8-9


In [8]:
df_product_info = pd.DataFrame(columns=['Product Name','Description', 'Product Description', 'ASIN','Manufacturer'])

In [9]:
df_product_info

,Product Name,Description,Product Description,ASIN,Manufacturer


In [10]:
number_of_products=200 #there are 426 products

i=1
# loop through each URL
for url in urls[:number_of_products]:
    # send a GET request to the URL
    response = requests.get(url, headers=headers)
    
    # check if the response status code is 200 (OK)
    if response.status_code == 200:
        # get the filename from the URL
        print('url {} successful'.format(i))
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        
        #PRODUCT NAME
        product_name = soup.find('span', attrs= {'id':"productTitle"})
        if product_name:
            product_name = product_name.text.strip()
        else:
            product_name=''
        #print(product_name)
        
        #DESCRIPTION
        desc = soup.find('ul', attrs={'class':"a-unordered-list a-vertical a-spacing-mini"})
        if desc:
            desc = desc.find_all('span', class_="a-list-item")
            description = ''
            for ele in desc:
                description = description+re.sub(r'\s+', ' ', ele.text).strip()+'. '
        else:
            description = ''
            
            
        #PRODUCT DESCRIPTION
        
        
        product_description = soup.find('div', attrs = {'id':'productDescription'})
        
        if product_description!=None:
            product_description = product_description.find('span')
            if product_description!=None:
                product_description = product_description.text.strip()
        else:
            product_description=''
            
        #PRODUCT ASIN
        product_ASIN=soup.find('ul', class_='a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list')
        product_ASIN2 = soup.find('table', attrs={'id':'productDetails_detailBullets_sections1'})
        
        prod_man = soup.find('div', attrs={'id':"detailBullets_feature_div"})
        if prod_man:
            prod_man = prod_man.find_all('span', attrs={"class":"a-list-item"})
            for ele in prod_man:
                if 'ASIN ' == re.sub(r'\s+', ' ', ele.text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')[0]:
                    product_ASIN = re.sub(r'\s+', ' ', ele.text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')[1]
                    break
        elif product_ASIN:
            product_ASIN = product_ASIN.text.replace('\u200e', ' ').replace('\u200f',' ').replace('\n',' ')
            product_ASIN = re.sub(r'\s+', ' ', product_ASIN).strip() # Remove unnecessary whitespace
            product_ASIN = product_ASIN.split(':')[4].strip()[:10]
        elif product_ASIN2:
            product_ASIN2 = product_ASIN2.find('td', attrs={'class':"a-size-base prodDetAttrValue"}).text
            product_ASIN = product_ASIN2
        else:
            product_ASIN=''
        #print(product_ASIN)
        
        #PRODUCT MANUFACTURER
        prod_man = soup.find('div', attrs={'id':"detailBullets_feature_div"})
        product_man2 = soup.find('table', attrs={'id':'productDetails_detailBullets_sections1'})
        if prod_man:
            prod_man = prod_man.find_all('span', attrs={"class":"a-list-item"})
            for ele in prod_man:
                if 'Manufacturer ' == re.sub(r'\s+', ' ', ele.text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')[0]:
                    product_manufacturer = re.sub(r'\s+', ' ', ele.text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')[1]
                    break
        elif product_man2:
            product_man2 = product_man2.find_all('td', attrs={'class':"a-size-base prodDetAttrValue"})
            product_details = []
            for ele in product_man2:
                product_details.append(ele.text)
            product_manufacturer = product_details[2]
        else:
            product_manufacturer=''
        
        row_dict = {'Product Name': product_name, 'Description':description, 'Product Description': product_description,'ASIN': product_ASIN, 'Manufacturer':product_manufacturer}
        df_product_info = df_product_info.append(row_dict, ignore_index=True)
        i+=1

    else:
        print(f'Error retrieving HTML content from {url}. Status code: {response.status_code}')

url 1 successful
url 2 successful
url 3 successful
url 4 successful
url 5 successful
url 6 successful
url 7 successful
url 8 successful
url 9 successful
url 10 successful
url 11 successful
url 12 successful
url 13 successful
url 14 successful
url 15 successful
url 16 successful
url 17 successful
url 18 successful
url 19 successful
url 20 successful
url 21 successful
url 22 successful
url 23 successful
url 24 successful
url 25 successful
url 26 successful
url 27 successful
url 28 successful
url 29 successful
url 30 successful
url 31 successful
url 32 successful
url 33 successful
url 34 successful
url 35 successful
url 36 successful
url 37 successful
url 38 successful
url 39 successful
url 40 successful
url 41 successful
url 42 successful
url 43 successful
url 44 successful
url 45 successful
url 46 successful
url 47 successful
url 48 successful
url 49 successful
url 50 successful
url 51 successful
url 52 successful
url 53 successful
url 54 successful
url 55 successful
url 56 successful
u

In [23]:
df_product_info.tail(50)

,Product Name,Description,Product Description,ASIN,Manufacturer
150,House of Quirk Baby Diaper Bag Maternity Backpack,DURABLE MATERIAL:Diaper bags made of environme...,,B07QV7SH4S,House of Quirk
151,Lunar's Unisex 50L Water Resistant Office/Trav...,MADE IN INDIA - This “MAKE IN INDIA” Laptop ba...,,B08NHL52MK,"Akshaya Rubbers, Hebbal Industrial Area, Mysore"
152,uppercase SealPro Professional Laptop Backpack...,Care Instructions: Wipe with Damp Cloth. Pleas...,None,B0BR179L65,Acefour Accessories Pvt. Ltd.
153,"STRABO Oslo 37 L Nylon Black 15.6"" inch Laptop...",Long Lasting Material & Attractive Design.. Sm...,"Strabo Oslo backpack, a spacious and durable b...",B09SB4FKGH,Acefour Accessories Pvt. Ltd.
154,AAKANKASHA LEATHER BOUTIQUE laptop and paper b...,Closure: Zipper. Large sized black and grey co...,This ergonomically designed travel backpack is...,B0BR5BJCG7,Acefour Accessories Pvt. Ltd.
155,HEROZ Hammer Unisex Nylon 45 L Travel Laptop B...,"Material: Nylon, Inner Material: Polyester, Ca...",,B07MG3F91S,33.7 x 23.5 x 53.3 Centimeters
156,Amazon Basics - Laptop Backpack/College Bag/Tr...,"Laptop backpack made from water-resistant, str...",,B09W4WB3K5,FLAB INTERNATIONAL PRIVATE LIMITED PLOT NO. 3...
157,Tinytot School Bag School Backpack College Bac...,For Girls & Women. Water Resistant.,"At Tinytot, we believe travel should be fun. W...",B09C5J2MN7,Tinytot
158,F Gear Tycoon 27 Liters Laptop Backpack Sch Ba...,Both Professional and Stylish. Large zippered ...,If you have loads to carry around your search ...,B071XJ6XV5,Uber Fashion Merchandise Pvt. Ltd.
159,Protecta The Professional Laptop Bag Briefcase...,Made with Love in India.. Constructed Using Wa...,,B07GNCRCP4,Protecta Inc


In [27]:
df_product_info.to_csv('Submission_Part_2.csv',index=False)

# Side work or rough work

In [24]:
pwd

'/Users/admin/Desktop/00Analystt.ai'

In [12]:
link=urls[0]
response = requests.get(link, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [13]:
desc = soup.find('ul', attrs={'class':"a-unordered-list a-vertical a-spacing-mini"})

In [14]:
desc = desc.find_all('span', class_="a-list-item")

In [15]:
description = ''
for ele in desc:
    description = description+re.sub(r'\s+', ' ', ele.text).strip()+'. '

In [16]:
description

'Care Instructions: Wipe with Damp Cloth. Handcrafted with care and made up of premium quality robust material. Padded anti-theft laptop compartment fits up to 15.6" Laptop and a tablet compartment; Zipper Pocket in front along with a slip In Pocket in front; Internal Organisers with a bottle holder on the side. Wrinkle free bag and 3x more water resistant with Rainproof zippers; Trolley sleeve to securely place the bags on the luggage; Comfortable Padded shoulder straps. GRS certified for sustainability; The GRS (Global Recycled Standard) is intended for use with any product that contains at least 50% Recycled Material (License No. CMPY-158596); 750 days of international warranty. '

In [17]:
prod_man = soup.find('div', attrs={'id':"detailBullets_feature_div"})

In [18]:
prod_man = prod_man.find_all('span', attrs={"class":"a-list-item"})

In [19]:
re.sub(r'\s+', ' ', prod_man[2].text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')

['Manufacturer ', ' Acefour Accessories Pvt. Ltd.']

In [20]:
for ele in prod_man:
    if 'ASIN ' == re.sub(r'\s+', ' ', ele.text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')[0]:
        print(re.sub(r'\s+', ' ', ele.text.replace('\u200f',' ').replace('\n',' ').replace(' \u200e', ' ')).strip().split(':')[1])

 B0BSH4H4DB


In [21]:
product_ASIN = soup.find('table', attrs={'id':'productDetails_detailBullets_sections1'})
product_ASIN.find_all('td', attrs={'class':"a-size-base prodDetAttrValue"})
product_details = []
for ele in product_ASIN.find_all('td', attrs={'class':"a-size-base prodDetAttrValue"}):
    product_details.append(ele.text)
product_details

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
product_ASIN = soup.find('table', attrs={'id':'productDetails_detailBullets_sections1'})
product_ASIN.find('td', attrs={'class':"a-size-base prodDetAttrValue"}).text

In [ ]:
product_description = soup.find('div', attrs = {'id':'productDescription'})
product_description = product_description.find('span')

In [ ]:
product_description!=None

In [ ]:
print(product_name)

In [ ]:
product_name = soup.find('span', class_='a-size-large product-title-word-break')
if product_name:
    product_name = product_name.text.strip()
else:
    product_name=''